# Setup
This notebook performs the following setup actions for this example use of Amazon SageMaker Feature Store:

1. Create online-only feature groups
2. Create an Amazon Kinesis data stream
3. Create an Amazon Kinesis Data Applications (KDA) application

### Get ARN's of Lambda functions from CloudFormation stack outputs
1. InvokeFraudEndpointLambdaARN
2. StreamingAggLambdaARN

In [98]:
STACK_NAME = 'FeatureStoreStack-Test'

In [99]:
import sys
import boto3

cf_client = boto3.client('cloudformation')

try:
    outputs = cf_client.describe_stacks(StackName=STACK_NAME)['Stacks'][0]['Outputs']
    for o in outputs:
        if o['OutputKey'] == 'IngestLambdaFunctionARN':
            lambda_to_fs_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionARN':
            lambda_to_model_arn = o['OutputValue']
        if o['OutputKey'] == 'PredictLambdaFunctionName':
            predict_lambda_name = o['OutputValue']

except:
    msg = f'CloudFormation stack {STACK_NAME} was not found. Please set the STACK_NAME properly and re-run this cell'
    sys.exit(ValueError(msg))

In [100]:
print(f'lambda_to_model_arn: {lambda_to_model_arn}')
print(f'lambda_to_fs_arn: {lambda_to_fs_arn}')
print(f'predict_lambda_name: {predict_lambda_name}')

lambda_to_model_arn: arn:aws:lambda:us-east-2:105242341581:function:InvokeFraudEndpointLambda-v4
lambda_to_fs_arn: arn:aws:lambda:us-east-2:105242341581:function:StreamingIngestAggFeatures-v4
predict_lambda_name: InvokeFraudEndpointLambda-v4


In [101]:
%store lambda_to_model_arn

Stored 'lambda_to_model_arn' (str)


In [102]:
%store predict_lambda_name

Stored 'predict_lambda_name' (str)


In [103]:
# to get the latest sagemaker python sdk
#!pip install -U sagemaker

In [104]:
from IPython.display import display_html
def restartkernel() :
    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
#restartkernel()

### Imports and other setup

In [105]:
from sagemaker import get_execution_role
import sagemaker
import boto3
import json

role = get_execution_role()
sm = boto3.Session().client(service_name='sagemaker')
smfs_runtime = boto3.Session().client(service_name='sagemaker-featurestore-runtime')

## Create online-only feature groups
When using Amazon SageMaker Feature Store, a core design decision is the definition of feature groups. For our credit card fraud detection use case, we have decided to use two of them:

1. `cc-agg-fg` - holds aggregate features that will be updated in near real-time (streaming ingestion)
2. `cc-agg-batch-fg` - holds aggregate features that will be updated in batch

Establishing a feature group is a one-time step and is done using the `CreateFeatureGroup` API. 

Feature groups can be created as **online-only**, **offline-only**, or both **online and offline**, which replicates updates from an online store to an offline store in Amazon S3. Since our focus in this example is on demonstrating the use of the feature store for online inference and streaming aggregation of features, we make each of our feature groups online-only.

In addition to a feature group name, we provide metadata about each feature in the group. We are using a json file to define the schema, but this is not a requirement. We use a schema file to demonstrate how you might capture the feature group definitions, enabling you to recreate them consistently as you move from a development environment to a test or production environment. In our schema file, we also highlight the record identifier and the event timestamp. All feature groups must have these two features, but you get to decide how to name them.

#### cc-agg-fg schema

In [106]:
!pygmentize schema/cc-agg-fg-schema.json

{
    "description": "Aggregated features for each credit card, batch ingestion nightly",
    "features": [
          {
              "name": "cc_num",
              "type": "bigint",
              "description": "Credit Card Number (Unique)"
          },
          {
              "name": "num_trans_last_10m",
              "type": "bigint",
              "description": "Aggregated Metric: Average number of transactions for the card aggregated by past 10 minutes"
          },
          {
              "name": "avg_amt_last_10m",
              "type": "double",
              "description": "Aggregated Metric: Average transaction amount for the card aggregated by past 10 minutes"
          },
          {
              "name": "trans_time",
              "type": "double",
              "description": "Required feature for event timestamp"
          }
      ],
    
      "record_identifier_feature_name": "cc_num",
      "event_time_feature_name": "trans_time",
      "tags": [{"Key": "Envir

#### cc-agg-batch-fg schema

In [107]:
!pygmentize schema/cc-agg-batch-fg-schema.json

{
    "description": "Aggregated features for each credit card, streamed intraday",
    
    "features": [
          {
              "name": "cc_num",
              "type": "bigint",
              "description": "Credit Card Number (Unique)"
          },
          {
              "name": "num_trans_last_1w",
              "type": "bigint",
              "description": "Aggregated Metric: Average number of transactions for the card aggregated by past 1 week"
          },
          {
              "name": "avg_amt_last_1w",
              "type": "double",
              "description": "Aggregated Metric: Average transaction amount for the card aggregated by past 1 week"
          },
          {
              "name": "trans_time",
              "type": "double",
              "description": "Required feature for event timestamp"
          }
      ],
    
      "record_identifier_feature_name": "cc_num",
      "event_time_feature_name": "trans_time",
      "tags": [{"Key": "Environment", "V

#### Utility functions to simplify creation of feature groups
`schema_to_defs` takes our schema file and returns feature definitions, and the names of the record identifier and event timestamp feature.

In [108]:
def schema_to_defs(filename):
    schema = json.loads(open(filename).read())
    
    feature_definitions = []
    
    for col in schema['Features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_definitions.append(feature)

    return feature_definitions, schema['record_identifier_feature_name'], schema['event_time_feature_name']

`schema_to_fg` creates a feature group from a schema file. If no s3 URI is passed, an online-only feature group is created.

In [109]:
from sagemaker import get_execution_role
import json

def create_feature_group_from_schema(filename, fg_name, role_arn=None, s3_uri=None):
    schema = json.loads(open(filename).read())
    
    feature_defs = []
    
    for col in schema['features']:
        feature = {'FeatureName': col['name']}
        if col['type'] == 'double':
            feature['FeatureType'] = 'Fractional'
        elif col['type'] == 'bigint':
            feature['FeatureType'] = 'Integral'
        else:
            feature['FeatureType'] = 'String'
        feature_defs.append(feature)

    record_identifier_name = schema['record_identifier_feature_name']
    event_time_name = schema['event_time_feature_name']

    if role_arn is None:
        role_arn = get_execution_role()

    if s3_uri is None:
        offline_config = {}
    else:
        offline_config = {'OfflineStoreConfig': {'S3StorageConfig': {'S3Uri': s3_uri}}}
        
    sm.create_feature_group(
        FeatureGroupName = fg_name,
        RecordIdentifierFeatureName = record_identifier_name,
        EventTimeFeatureName = event_time_name,
        FeatureDefinitions = feature_defs,
        Description = schema['description'],
        Tags = schema['tags'],
        OnlineStoreConfig = {'EnableOnlineStore': True},
        RoleArn = role_arn,
        **offline_config)

#### Create the two feature groups

In [110]:
create_feature_group_from_schema('schema/cc-agg-fg-schema.json', 'cc-agg-fg')

In [111]:
create_feature_group_from_schema('schema/cc-agg-batch-fg-schema.json', 'cc-agg-batch-fg')

#### Show that the feature store is aware of the new feature groups

In [112]:
sm.list_feature_groups()

{'FeatureGroupSummaries': [{'FeatureGroupName': 'transaction-feature-group-02-12-52-50',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-02-12-52-50',
   'CreationTime': datetime.datetime(2020, 12, 2, 13, 0, 43, 462000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'transaction-feature-group-01-19-18-28',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-01-19-18-28',
   'CreationTime': datetime.datetime(2020, 12, 1, 19, 18, 45, 59000, tzinfo=tzlocal()),
   'FeatureGroupStatus': 'Created',
   'OfflineStoreStatus': {'Status': 'Active'}},
  {'FeatureGroupName': 'transaction-feature-group-01-12-29-08',
   'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/transaction-feature-group-01-12-29-08',
   'CreationTime': datetime.datetime(2020, 12, 1, 12, 30, 53, 245000, tzinfo=tzlocal(

#### Describe each feature group
Note that each feature group gets its own ARN, allowing you to manage IAM policies that control access to individual feature groups. The feature names and types are displayed, and the record identifier and event time features are called out specifically. Notice that there is only an `OnlineStoreConfig` and no `OfflineStoreConfig`, as we have decided not to replicate features offline for these groups.

In [113]:
sm.describe_feature_group(FeatureGroupName='cc-agg-fg')

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/cc-agg-fg',
 'FeatureGroupName': 'cc-agg-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_10m', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_10m', 'FeatureType': 'Fractional'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2020, 12, 7, 21, 43, 2, 844000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'RoleArn': 'arn:aws:iam::105242341581:role/service-role/AmazonSageMaker-ExecutionRole-20201028T113576',
 'FeatureGroupStatus': 'Creating',
 'Description': 'Aggregated features for each credit card, batch ingestion nightly',
 'ResponseMetadata': {'RequestId': 'fe42fe73-742f-4bf2-adbd-f1e23a37e6b1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'fe42fe73-742f-4bf2-

In [114]:
sm.describe_feature_group(FeatureGroupName='cc-agg-batch-fg')

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-2:105242341581:feature-group/cc-agg-batch-fg',
 'FeatureGroupName': 'cc-agg-batch-fg',
 'RecordIdentifierFeatureName': 'cc_num',
 'EventTimeFeatureName': 'trans_time',
 'FeatureDefinitions': [{'FeatureName': 'cc_num', 'FeatureType': 'Integral'},
  {'FeatureName': 'num_trans_last_1w', 'FeatureType': 'Integral'},
  {'FeatureName': 'avg_amt_last_1w', 'FeatureType': 'Fractional'},
  {'FeatureName': 'trans_time', 'FeatureType': 'Fractional'}],
 'CreationTime': datetime.datetime(2020, 12, 7, 21, 43, 7, 535000, tzinfo=tzlocal()),
 'OnlineStoreConfig': {'EnableOnlineStore': True},
 'RoleArn': 'arn:aws:iam::105242341581:role/service-role/AmazonSageMaker-ExecutionRole-20201028T113576',
 'FeatureGroupStatus': 'Creating',
 'Description': 'Aggregated features for each credit card, streamed intraday',
 'ResponseMetadata': {'RequestId': 'd78555a8-d080-4ef3-91e3-9ff22bf67cd4',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'd78555a8-d080-4

## Create an Amazon Kinesis Data Stream

In [117]:
kinesis_client = boto3.client('kinesis')

In [118]:
kinesis_client.create_stream(StreamName='cc-stream', ShardCount=1)

{'ResponseMetadata': {'RequestId': 'e96fdb0a-b488-2f24-b127-7f210a60c381',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'e96fdb0a-b488-2f24-b127-7f210a60c381',
   'x-amz-id-2': 'k+LYJZHCtftefxzaNnDv7gl72z6SRjqdFKUafzX5TD/jsvZCVaMytIshZltbx9buArXPHY0c5+jjaxC0McVNgqepdlt/5VYM',
   'date': 'Mon, 7 Dec 2020 21:43:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [119]:
kinesis_client.list_streams()

{'StreamNames': ['cc-stream'],
 'HasMoreStreams': False,
 'ResponseMetadata': {'RequestId': 'c63785b5-9164-6fed-9e7f-219e2f8c8348',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c63785b5-9164-6fed-9e7f-219e2f8c8348',
   'x-amz-id-2': 'lcs7POP0u+s5gL3O3X42nndhDbix8XjtcrlmyS8ChQe4x6DIOHNGOmZi4ftQJirRobrDp4hZoNmw4cuBog4KVHXyVACC1v+B',
   'date': 'Mon, 7 Dec 2020 21:43:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '52'},
  'RetryAttempts': 0}}

In [120]:
kinesis_client.describe_stream(StreamName='cc-stream')

{'StreamDescription': {'StreamName': 'cc-stream',
  'StreamARN': 'arn:aws:kinesis:us-east-2:105242341581:stream/cc-stream',
  'StreamStatus': 'CREATING',
  'Shards': [],
  'HasMoreShards': False,
  'RetentionPeriodHours': 24,
  'StreamCreationTimestamp': datetime.datetime(2020, 12, 7, 21, 43, 7, tzinfo=tzlocal()),
  'EnhancedMonitoring': [{'ShardLevelMetrics': []}],
  'EncryptionType': 'NONE'},
 'ResponseMetadata': {'RequestId': 'ea1d8ced-a665-b4d0-b255-28c6188d5875',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'ea1d8ced-a665-b4d0-b255-28c6188d5875',
   'x-amz-id-2': 'gwvKs1yLHIIboc+tW8nF8lYUCpQIbLJiRzc7bmvvt95W1PQWNydOuIl/C5tSoyefYXyPL+5uCAGhtKSktl+kLtQidTho0FTA',
   'date': 'Mon, 7 Dec 2020 21:43:07 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '316'},
  'RetryAttempts': 0}}

In [121]:
import time
active_stream = False
while not active_stream:
    status = kinesis_client.describe_stream(StreamName='cc-stream')['StreamDescription']['StreamStatus']
    if (status == 'CREATING'):
        print('Waiting for the Kinesis stream to become active...')
        time.sleep(20)  
    elif (status == 'ACTIVE'): 
        active_stream = True
        print('ACTIVE')

Waiting for the Kinesis stream to become active...
ACTIVE


In [122]:
stream_arn = kinesis_client.describe_stream(StreamName='cc-stream')['StreamDescription']['StreamARN']

In [123]:
stream_arn

'arn:aws:kinesis:us-east-2:105242341581:stream/cc-stream'

## Map the Kinesis stream as an event source for Lambda fraud detection

In [124]:
lambda_client = boto3.client('lambda')

lambda_client.create_event_source_mapping(EventSourceArn=stream_arn,
                                          FunctionName=lambda_to_model_arn,
                                          StartingPosition='LATEST',
                                          Enabled=True,
                                          MaximumRecordAgeInSeconds=60*10
                                          ) #DestinationConfig would handle discarded records

{'ResponseMetadata': {'RequestId': 'ca4c0ee4-589f-4a0d-a4c2-10d84da5f964',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Mon, 07 Dec 2020 21:43:28 GMT',
   'content-type': 'application/json',
   'content-length': '714',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ca4c0ee4-589f-4a0d-a4c2-10d84da5f964'},
  'RetryAttempts': 0},
 'UUID': '35d1bb0b-c4ae-4193-9820-6b8d598251ef',
 'BatchSize': 100,
 'MaximumBatchingWindowInSeconds': 0,
 'ParallelizationFactor': 1,
 'EventSourceArn': 'arn:aws:kinesis:us-east-2:105242341581:stream/cc-stream',
 'FunctionArn': 'arn:aws:lambda:us-east-2:105242341581:function:InvokeFraudEndpointLambda-v4',
 'LastModified': datetime.datetime(2020, 12, 7, 21, 43, 28, 233000, tzinfo=tzlocal()),
 'LastProcessingResult': 'No records processed',
 'State': 'Creating',
 'StateTransitionReason': 'User action',
 'DestinationConfig': {'OnFailure': {}},
 'MaximumRecordAgeInSeconds': 600,
 'BisectBatchOnFunctionError': False,
 'MaximumRetryAttempts': -1}

## Create an Amazon Kinesis Data Applications (KDA) application

In [125]:
kda_client = boto3.client('kinesisanalytics')

In [126]:
sql_code = 'CREATE OR REPLACE STREAM "DESTINATION_SQL_STREAM" (\n' + \
                '"cc_num"              BIGINT,\n' + \
                '"num_trans_last_10m"  SMALLINT,\n' + \
                '"avg_amt_last_10m"    REAL\n);\n\n' + \
            'CREATE OR REPLACE PUMP "STREAM_PUMP" AS\n' + \
            'INSERT INTO "DESTINATION_SQL_STREAM"\n' + \
                'SELECT STREAM "cc_num", \n' + \
                    'COUNT(*) OVER LAST_10_MINUTES, \n' + \
                    'AVG("amount") OVER LAST_10_MINUTES\n' + \
                    'FROM "SOURCE_SQL_STREAM_001"\n' + \
                    'WINDOW LAST_10_MINUTES AS (\n' + \
                        'PARTITION BY "cc_num"\n' + \
                        'RANGE INTERVAL \'10\' MINUTE PRECEDING);\n'

In [127]:
kda_inputs = [{
                'NamePrefix': 'SOURCE_SQL_STREAM',
                'KinesisStreamsInput': {
                       'ResourceARN': stream_arn,
                       'RoleARN': role
                },
                'InputSchema': {
                      'RecordFormat': {
                          'RecordFormatType': 'JSON',
                          'MappingParameters': {
                              'JSONMappingParameters': {
                                  'RecordRowPath': '$'
                              }
                          },
                      },
                      'RecordEncoding': 'UTF-8',
                      'RecordColumns': [
                          {'Name': 'cc_num',  'Mapping': '$.cc_num',   'SqlType': 'DECIMAL(1,1)'},
                          {'Name': 'merchant','Mapping': '$.merchant', 'SqlType': 'VARCHAR(64)'},
                          {'Name': 'amount', 'Mapping': '$.amount', 'SqlType': 'REAL'},
                          {'Name': 'zip_code', 'Mapping': '$.zip_code', 'SqlType': 'INTEGER'}
                      ]
                }
              }                         
             ]

<h3> Create Kinesis Data Analytics Application </h3>

We first lookup Lambda ARNs from CloudFormation output, then create a Kinesis Data Analytics application that connects its output to the Streaming Lambda. This Lambda will ingest the records and write them to the SageMaker Feature Group.

In [128]:
kda_outputs = [{'LambdaOutput': {'ResourceARN': lambda_to_fs_arn, 'RoleARN': role},
                'Name': 'DESTINATION_SQL_STREAM',
                'DestinationSchema': {'RecordFormatType': 'JSON'}}]

In [129]:
kda_outputs

[{'LambdaOutput': {'ResourceARN': 'arn:aws:lambda:us-east-2:105242341581:function:StreamingIngestAggFeatures-v4',
   'RoleARN': 'arn:aws:iam::105242341581:role/service-role/AmazonSageMaker-ExecutionRole-20201028T113576'},
  'Name': 'DESTINATION_SQL_STREAM',
  'DestinationSchema': {'RecordFormatType': 'JSON'}}]

In [130]:
kda_client.create_application(ApplicationName='cc-agg-app', 
                              Inputs=kda_inputs,
                              Outputs=kda_outputs,
                              ApplicationCode=sql_code)

{'ApplicationSummary': {'ApplicationName': 'cc-agg-app',
  'ApplicationARN': 'arn:aws:kinesisanalytics:us-east-2:105242341581:application/cc-agg-app',
  'ApplicationStatus': 'READY'},
 'ResponseMetadata': {'RequestId': '8ea4b784-74fa-4a8b-8b2c-f8b62b78ef8a',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '8ea4b784-74fa-4a8b-8b2c-f8b62b78ef8a',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '173',
   'date': 'Mon, 07 Dec 2020 21:43:28 GMT'},
  'RetryAttempts': 0}}

In [131]:
kda_client.describe_application(ApplicationName='cc-agg-app')

{'ApplicationDetail': {'ApplicationName': 'cc-agg-app',
  'ApplicationARN': 'arn:aws:kinesisanalytics:us-east-2:105242341581:application/cc-agg-app',
  'ApplicationStatus': 'READY',
  'CreateTimestamp': datetime.datetime(2020, 12, 7, 21, 43, 28, tzinfo=tzlocal()),
  'LastUpdateTimestamp': datetime.datetime(2020, 12, 7, 21, 43, 28, tzinfo=tzlocal()),
  'InputDescriptions': [{'InputId': '1.1',
    'NamePrefix': 'SOURCE_SQL_STREAM',
    'InAppStreamNames': ['SOURCE_SQL_STREAM_001'],
    'KinesisStreamsInputDescription': {'ResourceARN': 'arn:aws:kinesis:us-east-2:105242341581:stream/cc-stream',
     'RoleARN': 'arn:aws:iam::105242341581:role/service-role/AmazonSageMaker-ExecutionRole-20201028T113576'},
    'InputSchema': {'RecordFormat': {'RecordFormatType': 'JSON',
      'MappingParameters': {'JSONMappingParameters': {'RecordRowPath': '$'}}},
     'RecordEncoding': 'UTF-8',
     'RecordColumns': [{'Name': 'cc_num',
       'Mapping': '$.cc_num',
       'SqlType': 'DECIMAL(1,1)'},
      {'N

In [132]:
kda_client.start_application(ApplicationName='cc-agg-app',
                             InputConfigurations=[{'Id': '1.1',
                                                   'InputStartingPositionConfiguration': 
                                                     {'InputStartingPosition':'NOW'}}])

{'ResponseMetadata': {'RequestId': '2ea3510d-0d0d-410d-9c38-80dadf2725f0',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2ea3510d-0d0d-410d-9c38-80dadf2725f0',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'date': 'Mon, 07 Dec 2020 21:43:28 GMT'},
  'RetryAttempts': 0}}

# NOTES before finalizing
This nb needs both KinesisFullAccess and KinesisAnalyticsFullAccess

Role passed to create KDA app needs Lambda execution

nb role needs trust relationship with kinesisanalytics.amazonaws.com for passing role

In [133]:
print(f'boto3: {boto3.__version__}')


boto3: 1.16.19
